In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

# 3.0 Performing Sequencial Feature Selection 

## Notebook Setup: Imports and Configuration

In [ ]:
# ─────────────────────────────────────────────────────────────
#  Core Python & System Libraries
# ─────────────────────────────────────────────────────────────
import os
import sys
import time
import copy
import re
import itertools as it
from pprint import pprint

# ─────────────────────────────────────────────────────────────
#  Numerical & Data Manipulation
# ─────────────────────────────────────────────────────────────
import numpy as np
import pandas as pd

# ─────────────────────────────────────────────────────────────
#  Sklearn - ML Models & Metrics
# ─────────────────────────────────────────────────────────────
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline

# ─────────────────────────────────────────────────────────────
#  Plotting & Visualization
# ─────────────────────────────────────────────────────────────
# Matplotlib
import matplotlib.pyplot as plt
from scipy.stats import norm

# Plotly
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, plot
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.express as px
import plotly.io as pio

# ─────────────────────────────────────────────────────────────
#  Notebook Environment Settings
# ─────────────────────────────────────────────────────────────
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

init_notebook_mode(connected=True)  # For plotly offline mode

# Pandas display settings
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("mode.chained_assignment", None)
pd.options.display.max_colwidth = 200

# ─────────────────────────────────────────────────────────────
#  Warnings Configuration
# ─────────────────────────────────────────────────────────────
import warnings
from scipy.linalg import LinAlgWarning
from warnings import simplefilter

warnings.filterwarnings(action="ignore", category=LinAlgWarning, module="sklearn")
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

# ─────────────────────────────────────────────────────────────
#  Progress Bar Utility
# ─────────────────────────────────────────────────────────────
from tqdm.notebook import tqdm  # Notebook-friendly progress bar
from tqdm import trange

# ─────────────────────────────────────────────────────────────
#  Project-Specific Imports
# ─────────────────────────────────────────────────────────────
# Append project src/ directory to path
sys.path.append(os.path.abspath(os.path.join('..','..', 'src')))

from vis import *
from ml import *
from settings import *



## Loading and Preparing DFT Energy Datasets with 10-fold Cross Validation

In [ ]:
data_dir = "../../../data/external/dacs_energies_out"

# Define the file names and corresponding variable names
file_variables = {
    "Edacs_dft.pkl": "Edacs_dft",
    "Edft_din6_as_df.pkl":"Edft_din6_as_df",
    "Edft_din6_s_df.pkl":"Edft_din6_s_df",
    "Edft_din4_x2_df.pkl":"Edft_din4_x2_df",
    "Edft_din6_df.pkl":"Edft_din6_df",
    "Edft_din6_s_din4_x2_df.pkl":"Edft_din6_s_din4_x2_df",
    "Edft_din6_as_din4_x2_df.pkl":"Edft_din6_as_din4_x2_df",
    "Edft_balanced_df.pkl": "Edft_balanced_df"
    } 


# Create an empty dictionary to store the DataFrames
data_frames = {}

# Iterate over the file_variables dictionary
for file_name, variable_name in file_variables.items():
    file_path = os.path.join(data_dir, file_name)
    data_frame = pd.read_pickle(file_path)
    data_frames[variable_name] = data_frame


cv_setup = {"cv_type": "kfold", "cv_spec": 10}

# Create a dictionary to store the dataframes for easy access
dataframe_dict = {}

# Iterate over the data_frames dictionary
for key, df in data_frames.items():
    # Apply the add_cv_columns function to each dataframe
    kfold_df = add_cv_columns(df_in=df, cv_setup=cv_setup)
    # Create a variable name based on the original dataframe name with "_kfold" appended
    var_name = key + "_kfold"
    # Assign the kfold dataframe to the dynamically
    globals()[var_name] = kfold_df
    
    # Store the kfold dataframe in the dataframe dictionary
    dataframe_dict[var_name] = kfold_df



Edacs_dft_df_kfold = dataframe_dict["Edacs_dft_kfold"]
Edft_din6_as_df_kfold = dataframe_dict["Edft_din6_as_df_kfold"]
Edft_din6_s_df_kfold = dataframe_dict["Edft_din6_s_df_kfold"]
Edft_din4_x2_df_kfold = dataframe_dict["Edft_din4_x2_df_kfold"]
Edft_din6_df_kfold = dataframe_dict["Edft_din6_df_kfold"]
Edft_din6_s_din4_x2_df_kfold = dataframe_dict["Edft_din6_s_din4_x2_df_kfold"]
Edft_din6_as_din4_x2_df_kfold = dataframe_dict["Edft_din6_as_din4_x2_df_kfold"]
Edft_balanced_df_kfold= dataframe_dict["Edft_balanced_df_kfold"]

## Feature Space and Target Specification

In [ ]:
# ───────────────
# TARGET VARIABLE
# ───────────────
target = "Eads"


# ───────────────
# METAL FEATURES 
# ───────────────
metal_features = [
    "atomic_mass",
    "vdw_radius",
    "r_cov_sb",
    "r_cov_db",
    "dipole_polarizability",
    "ionic_radii_crystals",
    "d_center_sp",
    "Paul_electroneg",
    "MB_electroneg",
    "electron_affinity",
    "covalent_radius",
    "atomic_number",
    "Ion_energ_I",
    "Ion_energ_II",
    "Zung_radius",
    "Coh_radius",
    "Waber_radius",
    "mied_param_h",
    "mied_param_phi",
    "HOMO ",
    "LUMO",
    "mag_moment_bulk_d",
    " E_Fermi",
    "E_Fermi2"
]
 
# ───────────────
# CAVITY FEATURES
# ───────────────
cavity_features = [
    "ncoord",
    "number_hetero",
    "number_C",
    "frac_hetero",
    "frac_C",
    "number_hetero_six",
    "frac_hetero_six",
    "number_hetero_five",
    "frac_hetero_five",
    'delta_min_ds', 
    'delta_max_ds',
    'fermi_energy_cavity',
    'surface'
]



# ───────────────────────────────────────────────
# ELECTRONEGATIVITIES AND ATOMIC RADIUS FEATURES
# ───────────────────────────────────────────────
en_features = ['min(en)', 'mean(en)', 'max(en)', 'std(en)', 'sum(en)']
r_features = ['min(r)', 'mean(r)', 'max(r)', 'std(r)', 'sum(r)']

# ─────────────────────
# GEOMETRICAL FEATURES
# ───────────────────── 
posc_cd_features = ['min(posc_cavity_ds)', 'max(posc_cavity_ds)', 'mean(posc_cavity_ds)', 'std(posc_cavity_ds)',]
                    #'min(posc_cavity_ang)', 'max(posc_cavity_ang)', 'mean(posc_cavity_ang)','std(posc_cavity_ang)']


cont_cd_features = ['min(cont_cavity_ds)', 'max(cont_cavity_ds)', 'mean(cont_cavity_ds)', 'std(cont_cavity_ds)',]
                    #'min(cont_cavity_ang)','max(cont_cavity_ang)', 'mean(cont_cavity_ang)', 'std(cont_cavity_ang)']




# ──────────────────
# PRIMARY FEATURES
# ──────────────────
primary_features = metal_features + cavity_features + en_features + r_features + posc_cd_features + cont_cd_features


# ──────────────────
# FEATURE SETS
# ──────────────────
primary_feature_sets = {
    'metal': metal_features,
    'cavity': cavity_features,
    'full': primary_features,
}

##  Random Forest Regression: Full Feature Set Evaluation with K-Fold & LOOCV

This section evaluates the performance of a Random Forest model using the **full feature space**. It applies both **K-Fold Cross-Validation** and **Leave-One-Out Cross-Validation (LOOCV)** to estimate generalization error. The model is trained on the `Edft_balanced_df_kfold` dataset with `primary_features` and the defined target. The resulting RMSE values are recorded, and regression predictions are visualized using `plot_regr_train_test`.

In addition, structured NumPy arrays and empty DataFrames are initialized to later store model performance metrics (**R²**, **RMSE**, **MAE**, and their standard deviations) for **forward** and **backward feature selection** strategies under both K-Fold and LOOCV setups.

In [ ]:
# RF errors with full feature space, using K-Fold and LOOCV
full_rf_kfold_run = run_regr(
    df_in=Edft_balanced_df_kfold, #! Change it for other dataset
    ml_model=RandomForestRegressor(**rf_dict),
    ml_features=primary_features,
    ml_target=target,
)
full_rf_kfold_rmse = np.average(full_rf_kfold_run["error_dict"]["rmse_tests"])

full_rf_energy_fig_set = plot_regr_train_test(
        # result_dict=full_rf_kfold_run['result_dict'],
        # error_dict=full_rf_kfold_run['error_dict'],
        # column='metal',
        # showlegend=True,

        regr_dict=full_rf_kfold_run,
        color_column="metal",
        regr_layout=energy_layout,
        text_column="System",
        which_error="all",
        show_test_legend=False,
        show_train_legend=True,
    )

#full_rf_energy_fig_set.show()


full_rf_loocv_run = run_regr(
    df_in=Edft_balanced_df_kfold, #! Change it for other dataset
    ml_model=RandomForestRegressor(**rf_dict),
    ml_features=primary_features,
    ml_target=target,
)
full_rf_loocv_rmse = np.average(full_rf_loocv_run["error_dict"]["rmse_tests"])

full_rf_energy_fig_set = plot_regr_train_test(
        # result_dict=full_rf_loocv_run['result_dict'],
        # error_dict=full_rf_loocv_run['error_dict'],
        # column='metal',
        # showlegend=True,

        regr_dict=full_rf_loocv_run,
        color_column="metal",
        regr_layout=energy_layout,
        text_column="System",
        which_error="all",
        show_test_legend=False,
        show_train_legend=True,
    )


columns = ["feature", "r2", "r2_std", "rmse", "rmse_std", "mae", "mae_std", "eval_sets"]
#rf_array = np.zeros(shape=(len(full_primary_set), 8))


rf_array = np.zeros(shape=(len(primary_features) , 8))


model_evaluation_dfs = {
    "rf_forward_kfold": pd.DataFrame(data=copy.deepcopy(rf_array), columns=columns),
    "rf_forward_loocv": pd.DataFrame(data=copy.deepcopy(rf_array), columns=columns),
    "rf_backward_kfold": pd.DataFrame(data=copy.deepcopy(rf_array), columns=columns),
    "rf_backward_loocv": pd.DataFrame(data=copy.deepcopy(rf_array), columns=columns),
}


## Forward Feature Selection using K-Fold CV

This section implements a forward feature selection strategy using a Random Forest regressor with K-Fold cross-validation. At each step, the feature that yields the greatest improvement in model performance (lowest average test RMSE) is added to the feature set. The process continues until no further improvements can be made. The selected features and corresponding RMSE values are stored for later analysis.

In [ ]:
# %%capture

# Add primary_set always gets extended with the feature that leads to the largest performance gain
rf_forward_kfold_rmses = []
rf_forward_kfold_features = []
rf_forward_kfold_evaluated_sets = []

while True:

    try:

        # `Running` are the ones that get varied individually
        running_features = [_ for _ in primary_features if _ not in rf_forward_kfold_features]

        running_rmses = []
        running_models = []

        for running_feature in running_features:

            running_rf_set = run_regr(
                df_in=Edft_balanced_df_kfold, ml_model=RandomForestRegressor(**rf_dict), #! Change it for other dataset
                ml_features=rf_forward_kfold_features+[running_feature], ml_target=target,
            
            )

            running_rmses.append(np.mean(running_rf_set['error_dict']['rmse_tests']))

            print("Feature: {:<20s} & MAE: {:.6f} & RMSE: {:.6f}".format(
                running_feature,
                np.average(running_rf_set['error_dict']['mae_tests']),
                np.average(running_rf_set['error_dict']['rmse_tests']))
            )
        print("Best feature: {} -> Error: ".format(running_features[np.argmin(running_rmses)], min(running_rmses)))
        print('#'*80)
        
        rf_forward_kfold_features.append(running_features[np.argmin(running_rmses)])
        rf_forward_kfold_rmses.append(min(running_rmses))
        rf_forward_kfold_evaluated_sets.append(copy.deepcopy(rf_forward_kfold_features))

    except ValueError:
        break

model_evaluation_dfs['rf_forward_kfold']['feature'] = rf_forward_kfold_features
model_evaluation_dfs['rf_forward_kfold']['eval_sets'] = rf_forward_kfold_evaluated_sets

## Error Evolution During Forward Feature Selection (K-Fold CV)

This section evaluates the model performance (R², RMSE, MAE) for each feature subset obtained during forward selection. A Random Forest regressor is trained using K-Fold cross-validation on each subset. The mean and standard deviation of performance metrics are recorded and saved to a CSV file for further analysis or visualization.

In [ ]:
# Evaluate error course
for irf_forward_kfold_set, rf_forward_kfold_set in enumerate(model_evaluation_dfs['rf_forward_kfold']['eval_sets']):
    print(rf_forward_kfold_set)
    
    rf_set = run_regr(
        df_in=Edft_balanced_df_kfold, ml_model=RandomForestRegressor(**rf_dict),
        ml_features=rf_forward_kfold_set, ml_target=target
        )

    model_evaluation_dfs['rf_forward_kfold'].iloc[irf_forward_kfold_set, 1:-1] = [
        np.mean(rf_set['error_dict']['rsquared_fulls']), np.std(rf_set['error_dict']['rsquared_fulls']),
        np.mean(rf_set['error_dict']['rmse_tests']), np.std(rf_set['error_dict']['rmse_tests']),
        np.mean(rf_set['error_dict']['mae_tests']), np.std(rf_set['error_dict']['mae_tests']),
    ] 
    
model_evaluation_dfs['rf_forward_kfold'].head(30)
_ = model_evaluation_dfs['rf_forward_kfold'].to_csv('../../../data/sfs/rf_forward_dc_df_kfold.csv', sep=',', index=False) 